In [3]:
import numpy as np  
import pandas as pd  

In [6]:
data = pd.read_csv('../input/lg-propro/data.csv')
data['payment'].fillna(0,inplace=True)
data['payment'].describe()

count    815873.000000
mean          2.174848
std         121.434997
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max       25000.000000
Name: payment, dtype: float64

# 지금부터 할 일  ::  Serch data 처리 (encoding)
search data를 확인하고, 검색된 전적의 아이템 인코딩

그런데 유저 기준으로 하나? 특별히 선호하는 아이템?    
검색 데이터의 양은 현저히 적기 때문에 이것을 네거티브 샘플링하는데엔 문제가 있고, 선호를 반영하는 포지티브 샘플링으로는 의미가 있다.    
그렇다면 이것을 어떻게 반영해야 하는가... 

> 인코딩 방법 2가지

1. 전체 아이템 대비 / 검색된 적 있는 아이템 1, 검색된 적 없는 아이템 0    
    ㄴ class imbalance 심각 / 4만개 아이템 - history 등장 아이템 2만개 - 검색된 아이템 1977개 가량 
        
2. 검색된 적 있는 아이템 내 / 해당 유저에게 검색된 아이템 1, 검색되지 않은 아이템 0    
    ㄴ 그렇다면 검색 기록이 있는 명 644명 유저에 대해서만 샘플링을 진행한다는 것인가? <**무의미**하다.      
    
       
    
> 최종적으로 시도할 방법 
1. 검색된 아이템 중 most popular (value_counts) / 이건 일단 원핫은 아님 단순 집계!        
    
 ㄴ 근데 그럼 이걸 어떻게 모델 학습에 쓸 수 있지? 그냥 mp 목록 ?     
    
2. 인코딩 방법 중 어떤 걸 써도, 어차피 merge는 불가능하다. 그렇다면 1번으로 우선 진행.  
 ㄴ 4만개 중 검색된 적 있는 아이템. 일단 dict 처리 해두면, 검색 여부 0/1을 column으로 merge 가능 (나중에 CF 돌릴때 item정보로 반영가능할지도?)     
 ㄴ 어차피 interaction이 없는 아이템들이 너무나도 많다... 

In [14]:
search = pd.read_csv('../input/lgground/search_data.csv')
ser_for_oneh = search.drop_duplicates(subset=['album_id'])['album_id']
# 검색 횟수를 담을 게 아니라, 단순히 검색 여부만을 원핫으로 진행. 따라서 drop 해버리기 
ser_list=ser_for_oneh.tolist() #검색된 적이 있는 album_id 들을 담은 list (len 1977)

추천할 수 있는 모든 아이템 ID 목록은 어디서 확인할 수 있지?    

1.history와 watch 에 포함된 unique한 album_id 개수 다름 (한쪽에서만 등장하는 id값 있음)    
2. history와 meta에 포함된 unique한 album_id 개수 다름

In [4]:
# 데이터셋에 한번이라도 등장한 album id 리스트 만들기

history = pd.read_csv('../input/lgground/history_data.csv')
history_id = history['album_id'].unique().tolist()
watch = pd.read_csv('../input/lgground/watch_e_data.csv')
watch_id = watch['album_id'].unique().tolist()
meta = pd.read_csv('../input/lgground/meta_data.csv')
meta_id = meta['album_id'].unique().tolist()
all_albumID_list = list(set(history_id+watch_id+meta_id))
print(f"추천 가능한 아이템 개수 : {len(all_albumID_list)}개")
print(f"history+watch 로그에 등장한 적 없는 아이템 :  {len(all_albumID_list) - len(data['album_id'].unique())}개")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [34]:
arr = np.zeros((39875,1))
Hot_search = pd.DataFrame(arr,index=all_albumID_list)
Hot_search.loc[ser_list, 0] = 1
Hot_search.rename(columns={0 : 'search_one'},inplace=True)
Hot_search 

,search_one
0,0.0
1,0.0
2,0.0
3,1.0
4,0.0
...,...
39870,0.0
39871,0.0
39872,0.0
39873,0.0


# search MP 100

In [30]:
search_mp100 = pd.DataFrame(search.value_counts(['album_id'])).reset_index().rename(columns={0:'cnt'}).sort_values(by='cnt',ascending=False, ignore_index=True).head(100)
search_mp100

,album_id,cnt
0,347,118
1,4325,107
2,1880,86
3,2337,64
4,2484,60
...,...,...
95,2666,10
96,115,10
97,364,10
98,1894,10


1. buy data, search data 의 profile_id나 album id가 겹치는지 확인을 해야 함.    
ㄴ 겹치지 않으면 차라리 append하는 게 낫다.    
ㄴ 겹치면?     

2. buy 데이터 자체에 0이상의 값은 다 1로 대치, nan 0으로 대치    


3. 의문 : payment 여부에 대해서 인코딩해 column을 추가하는 게 과연 모델학습적으로 의미가 있는가 ?     
사실 우리가 payment를 못 버린 이유는 : 유료인 영상을 체크를 하고 싶었던 거죠. 시청에 그만큼 장벽이 있으니까.    
처음 제시한 solution - payment에 값이 찍힌 것을 1로 인코딩해서 표시합시다.     
    ㄴ 그런데 첫 구매 후에 특정기간동안 재생한 로그에 대해서는, 사실은 유료임에도 무료로 찍히고 있음 (정보값이 혼선)    
    ㄴ 최종적으로는 : dict 형태로 아이템:판매여부(1/0) 으로 만든다. 
    
4. 데이터셋을 하나로 만드려고 해서 생긴 문제 (submission dataset에 붙잡혀 있었음)      
    ㄴ user / item 데이터셋을 분리해 두는 것도 나쁘지 않을 듯 (ss_id, log_time같은 시계열적 속성을 모두 버린)      
    ㄴ 그냥 말 그대로의 유저정보/ 아이템 정보 